# Including Bus Routes

Purpose: Identify bus stop locations along routes in order to estimate the vehicle drive cycle.

Work Flow: Isolate bus stops along desired route. Calculate distance between route points and bus stops. 

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [3]:
import route_dynamics.route_elevation.base as base

import geopandas as gpd
import pandas as pd
import branca.colormap as cm
import folium
import rasterstats
import matplotlib.pyplot as plt

from folium.features import GeoJson
from shapely.geometry import mapping
from shapely.geometry import LineString
from shapely.geometry import Polygon
from rasterio.mask import mask
from geopy.distance import geodesic

First, input data and select bus route. 

In [4]:
routes_shp = '../data/six_routes.shp'
stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'
route_num = 45

Use base functions, `read_shape` and `extract_point_df` to create a dataframe of route points.  

In [5]:
route = base.read_shape(routes_shp, route_num)
points = base.extract_point_df(route)
points.head()

,coordinates
0,"(-122.30424440010776, 47.65042646627143)"
1,"(-122.30424998379348, 47.65041485164514)"
2,"(-122.30427784808448, 47.650359884437)"
3,"(-122.30431345405354, 47.65028282713853)"
4,"(-122.30435529243597, 47.650173880453444)"


Isolate bus stops that service desired route number.

In [6]:
stops = gpd.read_file(stops_shp)

First, need to remove some pesky 'none' types...

In [8]:
stops['ROUTE_LIST'].fillna(value=str(0), inplace=True)
#stops['ROUTE_LIST'].head(10)

In [7]:
stops_list = pd.DataFrame()
for i in range(0, len(stops)):
    
    if str(route_num) in (stops['ROUTE_LIST'][i]):
        for x in stops['ROUTE_LIST'][i].split(' '):
            if str(route_num) == x:
                stops_list = stops_list.append(stops.iloc[i])
            else:
                pass
        #stops_list = pd.concat([stops_list, stops.iloc[i]], axis = 0)
    else:
        pass

stops_list.head()

,ACCESSIBIL,ACCESSORY_,AUTH_CODE,AUTH_NAME,AWNING,BAY_CODE,BEARING_CO,BIKE_RACK,CF_CROSS_S,CF_DIST_FR,...,STOP_STATU,STOP_TYPE,TRANS_LINK,TRF_DISTRI,XCOORD,XCOORD_OFF,YCOORD,YCOORD_OFF,ZIPCODE,geometry
1139,YES,None,SEA,Seattle,NO,None,E,NO,Greenwood Ave N,89.7,...,ACT,REG,21581.0,NW,1265691.2,1265691.0,255613.3,255588.3,98103.0,POINT (-122.3549349116802 47.69052891200809)
1141,YES,None,SEA,Seattle,NO,None,E,NO,Fremont Ave N,74.9,...,ACT,REG,15929.0,NW,1267003.2,1267003.0,255580.3,255555.3,98103.0,POINT (-122.3496061788703 47.6905094486964)
1143,YES,None,SEA,Seattle,NO,None,E,NO,Aurora Ave N,34.7,...,ACT,REG,66216.0,NW,1268205.9,1268205.0,255549.6,255524.6,98103.0,POINT (-122.3447241811237 47.69049011915158)
1144,YES,None,SEA,Seattle,NO,None,E,NO,Stone Ave N,51.2,...,ACT,REG,149168.0,NW,1268998.5,1268998.0,255527.1,255502.1,98103.0,POINT (-122.3415031855857 47.69047109565094)
1146,YES,None,SEA,Seattle,NO,None,E,NO,Wallingford Ave N,40.1,...,ACT,REG,54504.0,NW,1270204.9,1270204.0,255494.0,255469.0,98103.0,POINT (-122.3366047660409 47.6904450549253)


Create a dataframe of bus stop points.

In [8]:
geometry = stops_list.geometry.values
xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame()
    xy_df['coordinates'] = xy
    
xy_df.head()

,coordinates
0,"(-122.35493491168019, 47.69052891200809)"
1,"(-122.34960617887032, 47.6905094486964)"
2,"(-122.34472418112374, 47.69049011915158)"
3,"(-122.34150318558568, 47.69047109565094)"
4,"(-122.33660476604088, 47.6904450549253)"


In [9]:
def make_lines(gdf, idx):
    
    coordinate_1 = gdf.loc[idx]['coordinates']
    coordinate_2 = gdf.loc[idx + 1]['coordinates']
    line = LineString([coordinate_1, coordinate_2])
    data = {'geometry':[line]}
    df_line = pd.DataFrame(data, columns = ['geometry'])

    return df_line


def make_route_lines(gdf):
    df_route = pd.DataFrame(columns = ['geometry'])
    for idx in range(len(gdf) - 1):
        df_linestring = make_lines(gdf, idx)
        df_route = pd.concat([df_route, df_linestring])
        
    gdf_route = gpd.GeoDataFrame(df_route)
    
    return gdf_route

In [10]:
gdf_route = make_route_lines(points)

In [11]:
gdf_stops = pd.DataFrame(xy, columns = ['Lat','Long'])

Show map:

In [12]:
UW_coords = [47.655548, -122.303200]
figure_size = folium.Figure(height = 400)
route_map = folium.Map(location = UW_coords, zoom_start = 12)
route_json = gdf_route.to_json()
route_layer = folium.GeoJson(route_json)
route_layer.add_child
route_map.add_child(route_layer)
route_map.add_to(figure_size)

fg = folium.FeatureGroup()
for i in range(len(gdf_stops)):
    fg.add_child(folium.Circle(location=[gdf_stops['Long'][i],gdf_stops['Lat'][i]], radius=8, color='red'))

route_map.add_child(fg)
route_map